In [1]:
import os
import pickle
from pymongo import MongoClient
import json
import xmlrpc.client

In [2]:
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['FCP'] 
collection = db['Bugs'] 

In [3]:
# Define the conversion functions
def convert_datetime(obj):
    if isinstance(obj, xmlrpc.client.DateTime):
        return obj.value  # Extract the value from DateTime object
    return obj

def convert_document(document):
    if isinstance(document, dict):
        return {k: convert_document(v) for k, v in document.items()}
    elif isinstance(document, list):
        return [convert_document(item) for item in document]
    else:
        return convert_datetime(document)

# Specify the directory path where the pickle files are located
pickle_dir = r'/Users/irene/Desktop/.bugs'

# Initialize a list to accumulate converted documents
converted_documents = []

# Iterate through each pickle file in the directory
for filename in os.listdir(pickle_dir):
    if filename.endswith('.pickle'):
        file_path = os.path.join(pickle_dir, filename)

        # Load the pickle file and convert it into a Python object
        with open(file_path, 'rb') as handle:
            obj = pickle.load(handle)

        # Convert the object to a compatible format
        converted_obj = convert_document(obj)

        # Append the converted document to the list
        converted_documents.append(converted_obj)

        # Insert documents in batches (e.g., 1000 documents per batch)
        if len(converted_documents) == 1000:
            # Serialize the batch as JSON
            json_data = json.dumps(converted_documents)

            # Insert the JSON data as documents into the MongoDB collection
            collection.insert_many(json.loads(json_data))

            # Clear the list for the next batch
            converted_documents = []

# Insert the remaining documents in the last batch
if converted_documents:
    # Serialize the remaining batch as JSON
    json_data = json.dumps(converted_documents)

    # Insert the JSON data as documents into the MongoDB collection
    collection.insert_many(json.loads(json_data))

# Close the MongoDB connection
client.close()
